In [1]:
import numpy as np  # 导入NumPy库用于数值计算
import torch  # 导入PyTorch库用于深度学习
import torch.nn as nn  # 导入PyTorch神经网络模块
import torch.optim as optim  # 导入PyTorch优化器模块
from torch.utils.data import Dataset, DataLoader  # 导入数据集和数据加载器类
import random  # 导入随机数生成模块
import time  # 导入时间模块用于计时
import torch.nn.functional as F  # 导入PyTorch函数式接口
from torch_geometric.data import Data  # 导入PyTorch Geometric数据类
from torch_geometric.nn import RGCNConv  # 导入关系图卷积网络层
import gc  # 导入垃圾回收模块
import psutil  # 导入系统资源监控模块

In [2]:
# 设置随机种子以确保结果可复现
def set_seed(seed=123):
    random.seed(seed)  # 设置Python随机种子
    np.random.seed(seed)  # 设置NumPy随机种子
    torch.manual_seed(seed)  # 设置PyTorch CPU随机种子
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)  # 设置当前GPU随机种子
        torch.cuda.manual_seed_all(seed)  # 设置所有GPU随机种子
        torch.backends.cudnn.deterministic = True  # 确保CUDA卷积操作的确定性
        torch.backends.cudnn.benchmark = False  # 禁用cudnn benchmark以减少显存使用
        torch.backends.cudnn.enabled = True  # 保持cudnn启用以获得更好的性能

set_seed()  # 调用函数设置随机种子

In [3]:
# 数据路径
TRAIN_FILE_PATH = "/Users/minkexiu/Downloads/GitHub/Tianchi_EcommerceKG_mac/originalData/OpenBG500/OpenBG500_train.tsv"  # 训练数据文件路径
TEST_FILE_PATH = "/Users/minkexiu/Downloads/GitHub/Tianchi_EcommerceKG/originalData/OpenBG500/OpenBG500_test.tsv"  # 测试数据文件路径
DEV_FILE_PATH = "/Users/minkexiu/Downloads/GitHub/Tianchi_EcommerceKG/originalData/OpenBG500/OpenBG500_dev.tsv" # 开发集路径
OUTPUT_FILE_PATH = "/Users/minkexiu/Downloads/GitHub/Tianchi_EcommerceKG/preprocessedData/OpenBG500_test.tsv"  # 输出结果文件路径

In [4]:
# 超参数
EMBEDDING_DIM = 200  # 实体和关系嵌入的维度
LEARNING_RATE = 0.001  # 学习率
WEIGHT_DECAY = 1e-5  # 权重衰减（L2正则化）
BATCH_SIZE = 256  # 训练批次大小（从1024减少到256以控制显存）
NEGATIVE_SAMPLES = 5  # 每个正样本对应的负样本数量

EPOCHS = 6  # 训练的epoch数量
MAX_LINES = None  # 限制训练数据的行数，None表示使用全部数据
max_head_entities = None  # 限制预测的头实体数量，None表示处理全部

# 内存和显存限制配置
MAX_GPU_MEMORY_GB = 32  # 最大GPU显存使用限制（GB）
MAX_CPU_MEMORY_GB = 16  # 最大CPU内存使用限制（GB）
GRADIENT_ACCUMULATION_STEPS = 4  # 梯度累积步数（用于保持有效批次大小）
MIXED_PRECISION = True  # 启用混合精度训练

# 学习率调度器参数
LR_DECAY_FACTOR = 0.5  # 学习率衰减因子
LR_DECAY_STEP = 50  # 每隔多少个epoch衰减一次学习率

In [5]:
# 数据加载类
class KnowledgeGraphDataset(Dataset):
    def __init__(self, file_path, is_test=False, is_dev=False, max_lines=None):
        self.is_test = is_test  # 标记是否为测试集
        self.is_dev = is_dev  # 标记是否为开发集
        self.triples = []  # 存储三元组数据的列表
        
        # 读取文件
        with open(file_path, 'r', encoding='utf-8') as f:
            line_count = 0  # 记录已读取的行数
            for line in f:
                if max_lines is not None and line_count >= max_lines:
                    break  # 如果达到最大行数限制，停止读取
                
                parts = line.strip().split('\t')  # 按制表符分割每行数据
                if is_test:
                    # 测试文件只有头实体和关系
                    if len(parts) >= 2:
                        h, r = parts[0], parts[1]  # 提取头实体和关系
                        self.triples.append((h, r, None))  # 将测试样本添加到列表，尾实体为None
                        line_count += 1  # 增加行数计数
                else:
                    # 训练和开发文件有完整的三元组
                    if len(parts) >= 3:
                        h, r, t = parts[0], parts[1], parts[2]  # 提取头实体、关系和尾实体
                        self.triples.append((h, r, t))  # 将完整三元组添加到列表
                        line_count += 1  # 增加行数计数
    
    def __len__(self):
        return len(self.triples)  # 返回数据集大小
    
    def __getitem__(self, idx):
        return self.triples[idx]  # 根据索引返回三元组数据

In [6]:
# 实体和关系映射管理器
class EntityRelationMapper:
    def __init__(self):
        self.entity_to_id = {}  # 实体到ID的映射字典
        self.id_to_entity = {}  # ID到实体的映射字典
        self.relation_to_id = {}  # 关系到ID的映射字典
        self.id_to_relation = {}  # ID到关系的映射字典
        self.entity_count = 0  # 实体数量
        self.relation_count = 0  # 关系数量
        
    def add_entity(self, entity):
        # 如果实体不存在于映射中，则添加
        if entity not in self.entity_to_id:
            self.entity_to_id[entity] = self.entity_count  # 设置实体到ID的映射
            self.id_to_entity[self.entity_count] = entity  # 设置ID到实体的映射
            self.entity_count += 1  # 实体计数加1
    
    def add_relation(self, relation):
        # 如果关系不存在于映射中，则添加
        if relation not in self.relation_to_id:
            self.relation_to_id[relation] = self.relation_count  # 设置关系到ID的映射
            self.id_to_relation[self.relation_count] = relation  # 设置ID到关系的映射
            self.relation_count += 1  # 关系计数加1
    
    def build_mappings(self, train_dataset, test_dataset, dev_dataset=None):
        # 从训练数据构建映射
        for h, r, t in train_dataset.triples:
            self.add_entity(h)  # 添加头实体到映射
            self.add_entity(t)  # 添加尾实体到映射
            self.add_relation(r)  # 添加关系到映射
        
        # 从测试数据构建映射（确保所有实体和关系都被包含）
        for h, r, _ in test_dataset.triples:
            self.add_entity(h)  # 添加头实体到映射
            self.add_relation(r)  # 添加关系到映射
        
        # 从开发集构建映射（确保所有实体和关系都被包含）
        if dev_dataset is not None:
            for h, r, t in dev_dataset.triples:
                self.add_entity(h)  # 添加头实体到映射
                self.add_entity(t)  # 添加尾实体到映射
                self.add_relation(r)  # 添加关系到映射

In [ ]:
# RGCN模型实现
class RGCNKG(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim=200, hidden_dim=100, num_bases=30):
        super(RGCNKG, self).__init__()  # 调用父类构造函数
        
        # 实体和关系嵌入层
        self.entity_embeddings = nn.Embedding(num_entities, embedding_dim)  # 实体嵌入层
        self.relation_embeddings = nn.Embedding(num_relations, embedding_dim)  # 关系嵌入层
        
        # RGCN卷积层 - 使用基分解来减少参数数量
        self.conv1 = RGCNConv(embedding_dim, hidden_dim, num_relations=num_relations * 2, num_bases=num_bases)  # 第一个RGCN卷积层
        self.conv2 = RGCNConv(hidden_dim, embedding_dim, num_relations=num_relations * 2, num_bases=num_bases)  # 第二个RGCN卷积层
        
        # 输出层用于评分函数
        self.score_layer = nn.Sequential(
            nn.Linear(embedding_dim * 3, embedding_dim),  # 线性层1：将头、关系、尾嵌入连接后的维度转换
            nn.ReLU(),  # 激活函数
            nn.Linear(embedding_dim, 1)  # 线性层2：输出最终得分
        )
        
        # 初始化权重
        nn.init.xavier_uniform_(self.entity_embeddings.weight.data)  # 使用Xavier均匀分布初始化实体嵌入权重
        nn.init.xavier_uniform_(self.relation_embeddings.weight.data)  # 使用Xavier均匀分布初始化关系嵌入权重
        
    def memory_efficient_forward(self, data, chunk_size=1000):
        """内存高效的正向传播"""
        # 使用模型自己的实体嵌入作为输入特征
        x = self.entity_embeddings.weight  # 获取实体嵌入权重
        edge_index = data.edge_index  # 获取边索引
        edge_type = data.edge_type  # 获取边类型
        
        # 分块处理大图
        num_nodes = x.size(0)  # 获取节点数量
        if num_nodes > chunk_size:
            # 对于大图，分块处理
            h = torch.zeros_like(x)  # 创建与x相同形状的零张量
            for i in range(0, num_nodes, chunk_size):
                end_idx = min(i + chunk_size, num_nodes)  # 计算当前块的结束索引
                mask = torch.zeros(num_nodes, dtype=torch.bool, device=x.device)  # 创建掩码
                mask[i:end_idx] = True  # 标记当前块的节点
                
                # 获取子图的边
                node_mask = torch.zeros(num_nodes, dtype=torch.bool, device=x.device)  # 创建节点掩码
                node_mask[i:end_idx] = True  # 标记当前块的节点
                edge_mask = node_mask[edge_index[0]] & node_mask[edge_index[1]]  # 计算边掩码
                
                if edge_mask.sum() > 0:
                    sub_edge_index = edge_index[:, edge_mask]  # 获取子图的边索引
                    sub_edge_type = edge_type[edge_mask]  # 获取子图的边类型
                    sub_x = x.clone()  # 复制输入特征
                    
                    # 应用RGCN卷积
                    sub_h = self.conv1(sub_x, sub_edge_index, sub_edge_type)  # 应用第一个卷积层
                    sub_h = F.relu(sub_h)  # 应用ReLU激活函数
                    sub_h = self.conv2(sub_h, sub_edge_index, sub_edge_type)  # 应用第二个卷积层
                    
                    h[i:end_idx] = sub_h[i:end_idx]  # 保存当前块的输出
                else:
                    h[i:end_idx] = x[i:end_idx]  # 如果没有边，直接使用原始嵌入
        else:
            # 对于小图，直接处理
            h = self.conv1(x, edge_index, edge_type)  # 应用第一个卷积层
            h = F.relu(h)  # 应用ReLU激活函数
            h = self.conv2(h, edge_index, edge_type)  # 应用第二个卷积层
        
        return h  # 返回更新后的实体嵌入
        
    def forward(self, data):
        # 使用模型自己的实体嵌入作为输入特征
        x = self.entity_embeddings.weight  # 获取实体嵌入权重
        edge_index = data.edge_index  # 获取边索引
        edge_type = data.edge_type  # 获取边类型
        
        # 根据实体数量选择处理方式
        if x.size(0) > 10000:
            # 对于大图，使用内存高效的正向传播
            print("大图，使用内存高效的正向传播")  # 打印提示信息
            h = self.memory_efficient_forward(data, chunk_size=1000)  # 调用内存高效的正向传播方法
        else:
            # 对于小图，直接处理
            print("小图，直接处理")  # 打印提示信息
            h = self.conv1(x, edge_index, edge_type)  # 应用第一个卷积层
            h = F.relu(h)  # 应用ReLU激活函数
            h = self.conv2(h, edge_index, edge_type)  # 应用第二个卷积层
        
        return h  # 返回更新后的实体嵌入
        
    def get_entity_embeddings(self):
        # 返回实体嵌入（用于预测）
        return self.entity_embeddings.weight  # 返回实体嵌入权重
        
    def get_relation_embeddings(self):
        # 返回关系嵌入
        return self.relation_embeddings.weight  # 返回关系嵌入权重
        
    def score_triple(self, h, r, t):
        # 评分函数：计算三元组的得分
        # 将头实体、关系、尾实体的嵌入连接起来
        combined = torch.cat([h, r, t], dim=1)  # 沿着维度1连接嵌入向量
        # 通过评分层计算得分
        score = self.score_layer(combined)  # 通过评分层计算得分
        return score  # 返回三元组得分

In [ ]:
# 自定义的批处理函数
def collate_fn(batch, mapper):
    h_batch = [item[0] for item in batch]  # 提取批次中的头实体列表
    r_batch = [item[1] for item in batch]  # 提取批次中的关系列表
    t_batch = [item[2] for item in batch]  # 提取批次中的尾实体列表
    
    # 批量转换为ID，减少循环开销
    h_ids = torch.tensor([mapper.entity_to_id[h] for h in h_batch])  # 将头实体转换为ID张量
    r_ids = torch.tensor([mapper.relation_to_id[r] for r in r_batch])  # 将关系转换为ID张量
    
    if t_batch[0] is not None:
        # 训练数据
        t_ids = torch.tensor([mapper.entity_to_id[t] for t in t_batch if t is not None])  # 将尾实体转换为ID张量
        return h_ids, r_ids, t_ids  # 返回头实体ID、关系ID和尾实体ID
    else:
        # 测试数据
        return h_ids, r_ids, None  # 返回头实体ID、关系ID，尾实体ID为None